In [ ]:
'you can download /kaggle/input/respiration-count-video/testOne.mp4 video to see output of algorithm  '

In [ ]:

!pip install mediapipe
import cv2
import os
import mediapipe as mp
import numpy as np
import time

countNonZero=[]

cap= cv2.VideoCapture(0)

object_tracker=cv2.createBackgroundSubtractorMOG2(history=100,varThreshold=10)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
forcc=cv2.VideoWriter_fourcc(*'XVID')
size = (frame_width, frame_height)

out=cv2.VideoWriter('output.mp4',forcc,20.0,(300,300))
Respiration_count=0
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  


  while cap.isOpened():
    
    success, image = cap.read()

    if success==True:

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        #image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if not(results.pose_landmarks):
            print(0)
            continue
        # Draw the pose annotation on the image.

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        chest_landmark_x=[]
        chest_landmark_y=[]
        for i in [11,12,24,23]:
            x,y=results.pose_landmarks.landmark[i].x,results.pose_landmarks.landmark[i].y
            h, w, c = image.shape
            cx, cy = int(x * w), int(y * h)
            chest_landmark_x.append(cx)
            chest_landmark_y.append(cy)
        image=cv2.GaussianBlur(image,(5,5),0)

        mask=object_tracker.apply(image)
        chest=mask[min(chest_landmark_y):max(chest_landmark_y),min(chest_landmark_x):max(chest_landmark_x)]
        print(chest)

        k=np.ones((2,2),np.uint8)
        _,THRESH_BINARY=cv2.threshold(chest,254,255,cv2.THRESH_BINARY)
        erode=cv2.erode(THRESH_BINARY,k,iterations=1)


        countNonZero.append( cv2.countNonZero(erode))

        if len(countNonZero)>=6:
                if countNonZero[-2]<100 and (countNonZero[-1]-countNonZero[-2]>100 ):# are according to the distance of the camera from the patient
                    Respiration_count+=1
                   
                   
        cv2.putText(image,'Number of Respiration ',(10,170),cv2.FONT_HERSHEY_PLAIN,1,(255, 0, 0),2) 

        cv2.putText(image,f'{Respiration_count}',(10,210),cv2.FONT_HERSHEY_PLAIN, 3,(255, 0, 0), 3)    
  
        cv2.imshow('mask',mask)
        cv2.imshow('dilation',chest)

        out.write(image)
        cv2.imshow('MediaPipe Pose',image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else :
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()
